## Numerical Methods
### Activity 4: Splines

Scipy has methods for interpolation we can use for splines

In [ ]:
from scipy import interpolate as itp 
import numpy as np
import matplotlib.pyplot as plt

# Consider the example from the class.

X = [3,4.5,7,9]
Y = [2.5,1,2.5,0.5]


# The interp1d method will take the X and Y data and return an interpolating spline function 
f = itp.interp1d(X, Y)
f2 = itp.interp1d(X, Y, kind='quadratic')


# f and f_2 are functions, and we can plot them using matplotlib
x_line = np.linspace(3,9,100, endpoint = True)
plt.plot(X, Y,  'o' , x_line, f(x_line), '--', x_line, f2(x_line), '-')
plt.legend(['data', 'linear', 'quadratic'], loc='best')
plt.show()



Scipy also has other ways to find splines. Here's another example. The splrep method calculates splines for $X$ and $Y$ with degree $k$. 

In [ ]:
spl = itp.splrep(X, Y, k = 2)

# If we print spl we see it's hard to interpret the output, though the value of k does appear at the end 
# (you can test this by changing k).
print(spl)



The method splev makes sense of the output of splrep and uses it to find the associated interpolating function. 

In [ ]:
plt.plot(X, Y, 'o', x_line,  itp.splev(x_line, spl), 'tab:green')
plt.show()





In this case, using interp1d and using splrep + splev give what looks like the same result, but they use different methods behind the scenes, and may not always produce exactly the same output. You can find some discussion of this [here](http://scipy-user.10969.n7.nabble.com/splrep-splev-versus-interp1d-td5775.html).

Looking carefully at the output above, we see that it is not the same as the interpolating function created in the slides for this data. We can see this from the fact that in the slides the first spline is a straight line, because we set $a_1 = 0$ as part of our method. So the techniques for calculating splines Scipy uses are not the same as the technique we used. If you dig far enough into the Scipy documentation you can find some references, but we won't do that here. 

To get a method exactly like the one from the slides we have to implement it ourselves.

In [ ]:
# This function returns the interpolating function created from splines of degree k. 
# This should work correctly for small values of k, e.g. = 1,2,3 etc. For large values linalg.solve doesn't work properly.
# x_points needs to be arranged in order of size for this to work properly.
def spline_int(x_points, y_points, k = 1):
    n = len(x_points) -1
    M = np.zeros(((k+1)*n - (k-1), (k+1)*n))
    Y = np.zeros((k+1)*n - (k-1))
    # Now we add rows to M corresponding to the condition that splines pass through points in the data.
    for i in range(0,n):
        for j in range(k+1):
            M[2*i,(k+1)*i+j] = x_points[i]**(k-j)
            M[2*i+1,(k+1)*i+j] = x_points[i+1]**(k-j)
        Y[2*i] = y_points[i]
        Y[2*i+1] = y_points[i+1]
    # now we add rows corresponding to the conditions on equality of derivatives
    # d will track which derivative we are taking
    for d in range(1, k):
        # i will track which knot we are using
        for i in range(0,n-1):
            # Note that there are k+1-d terms are in the dth derivative
            # We add entries to M for each j in this range 
            for j in range(k+1-d):
                M[2*n + (d-1)*(n-1) + i, (k+1)*i+j] = (np.math.factorial(k-j)/np.math.factorial(k-j-d))*x_points[i+1]**(k-d-j)
                M[2*n + (d-1)*(n-1) + i, (k+1)*i+j+k+1] = -(np.math.factorial(k-j)/np.math.factorial(k-j-d))*x_points[i+1]**(k-d-j)
    # For k > 1 we need to trim some columns off M to represent setting some parameters to 0 to make the system solvable.
    M = np.delete(M, np.s_[0:k-1], 1)
    # We create an array of the parameters we set to zero.
    params = np.zeros(k-1)
    params = np.concatenate((params,np.linalg.solve(M,Y)))
    # Now we have the parameters we will define the function that spline_int will return.
    # This function f, which given a suitable value of x will return the value of the interpolating 
    # function at x (using the splines defined using params as created above)
    def f(values):
        # To make drawing graphs easier we want f to take as inputs either numbers or arrays of numbers, so we have to 
        # code for two cases. This isn't very neat, but it seems to be how numpy does it internally for things like numpy.sin.
        # As an aside, numpy isn't coded in python, as native python is quite slow. 
        # Numpy, like many other Python libraries, is actually written in C, making it very fast.
        values = np.asarray(values)
        x_min = min(x_points)
        x_max = max(x_points)
        if values.ndim == 0:
            if values < x_min  or values > x_max :
                return 'out of range'
            else:
                # We have to find the interval in which 'values' lies in order to use the correct spline function.
                spline_num = 0
                for i in range(n):
                    if values > x_points[i]:
                        spline_num = i
                    else:
                        break
                result = 0
                for j in range(k+1):
                    result += params[(k+1)*spline_num + j]*(values**(k-j))
                return  result
        else:
            y = []
            for x in values:
                result = 0
                if x < x_min or x > x_max:
                    return 'out of range'
                else:
                    # We have to find the interval in which x lies in order to use the correct spline function.
                    spline_num = 0
                    for i in range(n):
                        if x > x_points[i]:
                            spline_num = i
                        else:
                            break                
                for j in range(k+1):
                    result += params[(k+1)*spline_num + j]*(x**(k-j))
                y.append(result)
            return y    
    return f


    

You can change the value of $k$ in the definition of y_line to draw splines with polynomials of different degrees. You can check the output for $k=1$ agrees with the output from the first cell.

In [ ]:
k = 1
x_point = 5

y_line = spline_int(X,Y,k=k)(x_line)
y_point = spline_int(X,Y,k=k)(x_point)
plt.plot(X, Y, 'o', x_line, y_line, 'tab:green')
print('Model predicts y = {} when x = {}'.format(y_point, x_point))
plt.show()

# You can experiment by using different (x,y) data and different values of k.

Given the values X_new and Y_new, write a function that will find the parameters of the linear splines $(a_1,b_1,...,a_n,b_n)$. You can use the code from the previous cell as a guide (you will need to make some changes), or you can use another method if you prefer.

In [ ]:
def find_params(x_points, y_points):
    #TODO
    return

You can test your method on some new data. For reference, I think the correct output should be [ -4.  10.  10. -18.  -2.  18.   0.  10.  -7.  45.] This tells us $a_0 = -4$, $b_0 = 10$, $a_1 = 10$, $b_1 = -18$ etc.

In [ ]:
X2 = [1,2,3,4,5,6]
Y2 = [6,2,12,10,10,3]

print(find_params(X2,Y2))